In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
dfpre=pd.read_csv(r'C:\luyan\paper_code\data\hourly_for_pre_wm.csv')



dfpre['thermal']=dfpre['thermal_type'].astype(str)
dfpre['schedule']=dfpre['schedule_type'].astype(str)
dfpre['daytype'].replace(['work','rest'],['1','0'],inplace=True)
dfpre['workday']=dfpre['daytype'].astype(str)
dfpre['hour']=dfpre['hour'].astype(str)





dfselx0=dfpre[['thermal','schedule','workday','tout','hour','id','state']]



id_total=dfpre['id'].drop_duplicates()
id_train_70=dfpre['id'].drop_duplicates().sample(frac=0.7,random_state=123)
id_test_30=pd.concat([id_total, id_train_70, id_train_70]).drop_duplicates(keep=False)
dfselxtrain0=pd.merge(id_train_70,dfselx0,on='id')



dfytrain=dfselxtrain0['state'].values
y_train=np.array(dfytrain)



dfselxtrain=dfselxtrain0[['thermal','schedule','workday','tout','hour']]


dfxtrain=dfselxtrain
dfxtrain=dfxtrain.to_dict(orient='record')
v = DictVectorizer(sparse=False)
dfxtrain= v.fit_transform(dfxtrain)
scaler=StandardScaler().fit(dfxtrain)
x_train=scaler.transform(dfxtrain)





column=v.get_feature_names()


rus = RandomUnderSampler(random_state=123)
x_resampled, y_resampled = rus.fit_sample(x_train, y_train)

f1_scorer = make_scorer(f1_score) 


param_test2={'n_estimators':range(100,1001,200),'learning_rate':[0.001, 0.01, 0.05, 0.1],'max_depth':range(3,11,2)}

other_params = {'min_child_weight': 5, 'seed': 0,
                            'subsample': 0.9, 'colsample_bytree': 0.7, 'gamma': 0.5, 'reg_alpha': 10, 'reg_lambda': 10}

model = XGBClassifier(**other_params)

gsearch2 = GridSearchCV(estimator=model,param_grid=param_test2,scoring=f1_scorer,cv=5,n_jobs=6)
gsearch2.fit(x_resampled, y_resampled)


print (gsearch2.best_params_, gsearch2.best_score_)

C:\Anaconda\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:04:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 900} 0.7166906389764274
